In [11]:
import numpy as np
#import torch
import random
import os

from dscribe.descriptors import SOAP
from ase.io import read
from ase import Atoms
from ase.build import molecule

In [2]:
from sklearn.kernel_ridge import KernelRidge
from matplotlib import pyplot
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
import sqlite3
import numpy as np
import csv
import math
from rdkit import Chem
from rdkit.Chem import MACCSkeys
from sklearn.model_selection import cross_validate

In [3]:
def parse_float(s: str) -> float:
    try:
        return float(s)
    except ValueError:
        base, power = s.split('*^')
        return float(base) * 10**float(power)

In [4]:
'''
xyz format
- line 1: number of atoms n
- line 2: scalar properties
- line 3, ..., n+1: element type, coordinates xyz, Mulliken partial charges on atoms
'''

# parse an xyz file, returns a result dictionary of a molecule
def parse_xyz(filename):
    num_atoms = 0
    scalar_properties = []
    atomic_symbols = []
    xyz = []
    charges = []
    smiles = ""
    with open(filename, 'r') as f:
        for line_num, line in enumerate(f):
            if line_num == 0:
                num_atoms = int(line)
            elif line_num == 1:
                scalar_properties = [parse_float(i) for i in line.split()[2:]]
            elif 2 <= line_num <= 1 + num_atoms:
                atom_symbol, x, y, z, charge = line.split()
                atomic_symbols.append(atom_symbol)
                xyz.append([parse_float(x), parse_float(y), parse_float(z)])
                charges.append(parse_float(charge))
            elif line_num == num_atoms + 3:
                smiles = str(line.split())

    result = {
        'num_atoms': num_atoms,
        'atomic_symbols': atomic_symbols,
        'pos': np.array(xyz),
        'charges': np.array(charges),
        'smiles': smiles
    }
    return result

In [5]:
# given 1 <= start <= 133885 and 1 <= end <= 133885 return array of soap 
# vectors for molecules start, start+1, ..., end
def build_soap(start, end):
    soaps = []
    for i in range(start, end+1):

        # create molecule object
        file_name = "../../data/dsgdb9nsd.xyz/dsgdb9nsd_" + \
            str(i).zfill(6) + ".xyz"
        molecule = parse_xyz(file_name)
        molecule_obj = Atoms(symbols=molecule["atomic_symbols"], positions=molecule["pos"])

        # set up soap descriptor
        species = set()
        species.update(molecule_obj.get_chemical_symbols())

        soap = SOAP(
            species=species,
            periodic=False,
            rcut=5,
            nmax=8,
            lmax=8,
            average="outer",
            sparse=False
        )
        feature_vector = soap.create(molecule_obj)
        soaps.append(feature_vector)
    return soaps
    

In [6]:
def build_soap2(size):
    molecules = []
    for i in range(1, 133885+1):
        # create molecule object
        file_name = "../../data/dsgdb9nsd.xyz/dsgdb9nsd_" + \
            str(i).zfill(6) + ".xyz"
        molecule = parse_xyz(file_name)
        molecule_obj = Atoms(symbols=molecule["atomic_symbols"], positions=molecule["pos"])
        molecules.append(molecule_obj)
    
    random.seed(4) 
    random.shuffle(molecules)
    molecules = molecules[:size]
    # create SOAP object
    soap = SOAP(
        species=["C", "H", "O", "N", "F"],
        periodic=False,
        rcut=5,
        nmax=12,
        lmax=9,
        average="outer",
        sparse=False
    )
    
    feature_vector = soap.create(molecules)
    return feature_vector
        
    

In [7]:
def transform(filepath, size):
    # parse gap data from qm9
    u0 = {}
    with open(filepath, newline = '') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            u0[row['mol_id']] = float(row['u0_atom'])
    u0_data = list(u0.values())
    random.seed(4) 
    random.shuffle(u0_data)
    u0_data = u0_data[:size]
    soaps = build_soap2(size)
    print(soaps.shape)
    
    u0_train, u0_test = np.array(u0_data[:int(size*.75)]), np.array(u0_data[int(size*.75):])
    soap_train, soap_test = np.array(soaps[:int(size*.75)],dtype=object), np.array(soaps[int(size*.75):],dtype=object)
    return soap_train, soap_test, u0_train, u0_test

In [15]:
def create_model(size, alpha):
    model = KernelRidge(alpha=alpha)
    filepath = os.path.join("..","..","data","qm9.csv")
    soap_train, soap_test, u0_train, u0_test = transform(filepath, size)
    model.fit(soap_train, u0_train)
    predict = model.predict(soap_test)
    
    r_sqr = r2_score(u0_test, predict)
    rmse = math.sqrt(mean_squared_error(u0_test, predict))
    mae = mean_absolute_error(u0_test, predict)
    #print("gap test: ", gap_test)
    #print("gap predict: ", predict)
    print("r^2: ",r_sqr)
    print("rmse: ", rmse)
    print("mae: ", mae)

In [16]:
# nmax = 12, lmax = 9, train: 500, 75/25 split, kernel test
if __name__ == "__main__":
    #alphas = [0.0001, 0.001, 0.01, 0.1, 1, 10]
    size = 667
    create_model(size, 1)

(667, 18300)
r^2:  0.9970193111678198
rmse:  12.275675417932092
mae:  9.124100137095171


In [40]:
# nmax = 12, lmax = 9, train: 1250, 75/25 split
if __name__ == "__main__":
    size = 1667
    create_model(size)

(1667, 18300)
r^2:  0.9985193972516689
rmse:  9.173373871674238
mae:  6.648644137837299


In [41]:
# nmax = 12, lmax = 9, train: 5000, 75/25 split
if __name__ == "__main__":
    size = 6667
    create_model(size)

(6667, 18300)
r^2:  0.9989911799849506
rmse:  7.502455096672679
mae:  5.441493048620963


In [42]:
# nmax = 12, lmax = 9, train: 12500, 75/25 split
if __name__ == "__main__":
    size = 16667
    create_model(size)

(16667, 18300)
r^2:  0.9992442410329899
rmse:  6.609000137945278
mae:  4.884883331961093


In [43]:
# nmax = 12, lmax = 9, train: 25000, 75/25 split
if __name__ == "__main__":
    size = 25000
    create_model(size)

(25000, 18300)
r^2:  0.9993076473488788
rmse:  6.228635284057679
mae:  4.516556656800054


In [11]:
soap_train, soap_test, u0_train, u0_test = transform("../../data/qm9.csv", 6667)
print("u0 train sd: ", np.std(u0_train))
print("u0 test sd: ", np.std(u0_test))


u0 train sd:  254.07177904440334
u0 test sd:  246.82420245375928


In [12]:
soap_train, soap_test, u0_train, u0_test = transform("../../data/qm9.csv", 16667)
print("u0 train sd: ", np.std(u0_train))
print("u0 test sd: ", np.std(u0_test))


u0 train sd:  242.20061297230475
u0 test sd:  239.86647643987908


In [13]:
soap_train, soap_test, u0_train, u0_test = transform("../../data/qm9.csv", 33333)
print("u0 train sd: ", np.std(u0_train))
print("u0 test sd: ", np.std(u0_test))


u0 train sd:  224.61148933084095
u0 test sd:  220.01239203266059
